# Notebook to insert recipes from csv into db

In [1]:
import psycopg2
import os
import pandas as pd

In [2]:
recipes_2 = pd.read_csv("../raw_data/recipe_list_2010_JamieOliver.csv")

In [9]:
recipes_2.head(5)

,title,total_time,yields,ingredients,instructions,image,calories,url,cuisine,dietary_restrictions,num_of_ingredients,difficulty
0,Strawberry Blast Smoothie,5,1 serving(s),"['1 frozen bananas, cut into small chunks', '½...",Place all ingredients into blender and blend o...,https://imagesvc.meredithcorp.io/v3/mm/image?u...,204.3 calories,https://www.allrecipes.com/recipe/258310/straw...,Not specified,Not specified,3,easy
1,Italian Bread Using a Bread Machine,200,2 serving(s),"['4 cups unbleached all-purpose flour', '1 tab...","Place flour, brown sugar, warm water, salt, ol...",https://imagesvc.meredithcorp.io/v3/mm/image?u...,104.5 calories,https://www.allrecipes.com/recipe/22823/italia...,Not specified,Not specified,9,hard
2,Banana Juice,10,2 serving(s),"['2 cups milk', '2 large ripe bananas', '2 tab...","Blend milk, bananas, pistachios, and honey in ...",https://imagesvc.meredithcorp.io/v3/mm/image?u...,320.3 calories,https://www.allrecipes.com/recipe/231608/banan...,Not specified,Not specified,4,easy
3,Melon Lime Cooler,40,4 serving(s),"['4 ½ cups cubed honeydew melon', '1 ½ cups li...",Place honeydew melon in a single layer on a ba...,https://imagesvc.meredithcorp.io/v3/mm/image?u...,156 calories,https://www.allrecipes.com/recipe/215354/melon...,Not specified,Not specified,4,medium
4,Grilled Brie and Pear Sandwich,15,1 serving(s),"['2 tablespoons butter, softened', '2 thick sl...",Generously butter one side of each slice of br...,https://imagesvc.meredithcorp.io/v3/mm/image?u...,670.3 calories,https://www.allrecipes.com/recipe/237472/grill...,Not specified,Not specified,7,medium


TODO: Ingredients are currently a string, for our database we need them to be a list
+ Strip the brackets at the end and beginning
+ replace quotation marks 
+ split the string on commas 
--> we have a list 

In [15]:
#This is what we insert into our db for ingredients 
recipes_2.iloc[1]["ingredients"][1:-1].replace("'", "").split(",")

['4 cups unbleached all-purpose flour',
 ' 1 tablespoon light brown sugar',
 ' 1 ⅓ cups warm water (110 degrees F/45 degrees C)',
 ' 1 ½ teaspoons salt',
 ' 1 ½ teaspoons olive oil',
 ' 1 (.25 ounce) package active dry yeast',
 ' 1 egg',
 ' 1 tablespoon water',
 ' 2 tablespoons cornmeal']

In [5]:
recipes_2.isnull().sum()

title                      0
total_time                 0
yields                     0
ingredients                0
instructions               0
image                      0
calories                   0
url                        0
cuisine                 4122
dietary_restrictions    4122
num_of_ingredients         0
dtype: int64

In [6]:
#Fill NaN of cuisine and dietary restrictions
recipes_2.fillna("Not specified", inplace=True)

In [8]:
#add column for difficulty
for index, row in recipes_2.iterrows():
    if row["total_time"] < 15:
        recipes_2.loc[index, "difficulty"] = "easy"
    elif row["total_time"] < 45:
        recipes_2.loc[index, "difficulty"] = "medium"
    else:
        recipes_2.loc[index, "difficulty"] = "hard" 

In [10]:
recipes_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4122 entries, 0 to 4121
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   title                 4122 non-null   object
 1   total_time            4122 non-null   int64 
 2   yields                4122 non-null   object
 3   ingredients           4122 non-null   object
 4   instructions          4122 non-null   object
 5   image                 4122 non-null   object
 6   calories              4122 non-null   object
 7   url                   4122 non-null   object
 8   cuisine               4122 non-null   object
 9   dietary_restrictions  4122 non-null   object
 10  num_of_ingredients    4122 non-null   int64 
 11  difficulty            4122 non-null   object
dtypes: int64(2), object(10)
memory usage: 386.6+ KB


## 1. Create a new table in our database with the new format (only once)

In [11]:
DB_PASSWORD = os.environ['DB_PASSWORD']

In [12]:
conn = psycopg2.connect(database="d1hsr1c7nk56dl", user = "iadkkqrgljveni", host = "ec2-3-230-61-252.compute-1.amazonaws.com", port = "5432", password=DB_PASSWORD)

In [13]:
cur = conn.cursor()

In [ ]:
#Creating the table only ONCE
cur.execute(
    '''CREATE TABLE COOKIT_RECIPES
    (ID INT PRIMARY KEY     NOT NULL,
    TITLE           TEXT    NOT NULL,
    DIFFICULTY      TEXT,
    PREPTIME        INT,
    NUMBER_OF_INGREDIENTS INT,
    INGREDIENTS     TEXT[]     NOT NULL,
    CUISINE         TEXT     NOT NULL,
    CALORIES        TEXT,
    LINK            TEXT       NOT NULL,
    PICTURE_URL     TEXT       NOT NULL,
    INSTRUCTIONS    TEXT);''')

In [ ]:
conn.commit()

## 2. Insert recipes into newly created table 

In [14]:
#Based on loaded dataframe
def insert_into_db(recipes):
    
    for index, row in recipes.iterrows():
        primary_key = index
        title = row["title"]
        difficulty = row["difficulty"]
        prep_time = row["total_time"]
        number_of_ingredients = row["num_of_ingredients"]
        ingredients = recipes.iloc[index]["ingredients"][1:-1].replace("'", "").split(",")
        cuisine = row["cuisine"]
        calories = row["calories"]
        url = row["url"]
        picture_url = row["image"]
        instructions = row["instructions"]

        
        query = """INSERT INTO COOKIT_RECIPES (ID, TITLE, DIFFICULTY, PREPTIME, NUMBER_OF_INGREDIENTS, 
        INGREDIENTS, CUISINE, CALORIES, LINK, PICTURE_URL, INSTRUCTIONS)
        VALUES(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""
        
        cur.execute(query, (primary_key, title, difficulty, prep_time, number_of_ingredients, ingredients, cuisine, calories, url, picture_url, instructions));
        conn.commit()

In [ ]:
insert_into_db(recipes_2)